In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import matplotlib

In [2]:
def MA_CROSSOVER(df : pd.DataFrame, lookback_1 : int, lookback_2 : int) :

    df = df.copy()
 
    df['SMA'] = df['Close'].rolling(lookback_1 - 1).mean().shift(1)
    df['LMA'] = df['Close'].rolling(lookback_2 - 1).mean().shift(1)

    Signal = pd.Series(np.full(len(df),np.nan), index = df.index)
    Signal.loc[df['SMA'] > df['LMA']] = 1
    Signal.loc[df['SMA'] < df['LMA']] = -1
    Signal = Signal.ffill()  

    ##
    counter = 0
    entry = Signal.diff()
    for val in entry : 
        if abs(val) == 2 :
            counter += 1
    if (lookback_1 == 73 ) and (lookback_2 == 76) : print(counter)
    ##
    return Signal

In [3]:
def OPTI_MACO(df : pd.DataFrame) :

    best_profitfactor = 0
    best_lookback_1 = -1
    best_lookback_2 = -1

    returns = np.log(df['Close']).diff().squeeze()  

    for lookback1 in range(9,169) :
        for lookback2 in range(9,169) :
            if (lookback1 < lookback2) : 
                
                signal = MA_CROSSOVER(df,lookback1, lookback2)
                SR = (signal * returns).dropna() 
                signal_profitfactor = SR[SR > 0].sum() / SR[SR < 0].abs().sum()
            
                if (signal_profitfactor > best_profitfactor) :
                    best_profitfactor = signal_profitfactor
                    best_lookback_1 = lookback1
                    best_lookback_2 = lookback2

                
                #plt.plot(SR.cumsum(), label = f" {lookback1} : {lookback2} ")
                #plt.legend(loc=2)
                #plt.show()
                

    return best_lookback_1, best_lookback_2, best_profitfactor


In [ ]:
symbol = 'GLD'

df = yf.download([symbol], start = '2016-01-31', end = '2020-01-31', interval="1d")

bestLB1, bestLB2, bestPF = OPTI_MACO(df)

SIG = MA_CROSSOVER(df, bestLB1, bestLB2)

df['returns'] = np.log(df['Close']).diff().shift(-1)
df['system_returns'] = df['returns']*SIG
plt.plot(np.exp(df['system_returns']).cumprod(), label = 'Rendement')
plt.show()

print(f"best lookback 1 is : {bestLB1}")
print(f"best lookback 2 is : {bestLB2}")
print(f"best profit factor is : {bestPF}")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [ ]:
symbol = 'BTC-USD'

df = yf.download([symbol], start = '2016-01-31', end = '2020-01-31', interval="1d")

bestLB1, bestLB2, bestPF = OPTI_MACO(df)

SIG = MA_CROSSOVER(df, bestLB1, bestLB2)

df['returns'] = np.log(df['Close']).diff().shift(-1)
df['system_returns'] = df['returns']*SIG
plt.plot(np.exp(df['system_returns']).cumprod(), label = 'Rendement')
plt.show()

print(f"best lookback 1 is : {bestLB1}")
print(f"best lookback 2 is : {bestLB2}")
print(f"best profit factor is : {bestPF}")

best lookback 1 is : 73
best lookback 2 is : 76
best profit factor is : 1.3485540555573976


symbol = 'BTC-USD'
d1 = yf.download(
  [symbol],
  start = "2016-01-31",
  end = "2020-01-31",
  interval="1d"
)

s = MA_CROSSOVER(d1,9, 21)

print(s)

r = np.log(d1['Close']).diff().shift(-1).squeeze()

print(r[r.isna()])

sr1 = (s * r).dropna()

print(sr1)

signal_profitfactor = sr1[sr1 > 0].sum() / sr1[sr1 < 0].abs().sum()
print(signal_profitfactor)

